In [ ]:
etherpad_url = "http://localhost:9001/"
pad1_title = "TestPad1"
pad2_title = "TestPad2"
renamed_pad_title = "TestPad3"
hash_tag_original = "#TestPad2"
hash_tag_renamed = "#TestPad3"
default_result_path = None
close_on_fail = False
transition_timeout = 10000

index_page = None
pad1_url = None
pad2_url = None

In [ ]:
import tempfile

work_dir = tempfile.mkdtemp()
if default_result_path is None:
    default_result_path = work_dir
work_dir

# ep_weave E2E Test - Multiple Pads

- Test data to prepare: Not required

In [ ]:
import importlib

import scripts.playwright
importlib.reload(scripts.playwright)

from scripts.playwright import *

await init_pw_context(close_on_fail=close_on_fail, last_path=default_result_path)

## Open etherpad_url and confirm the search box is editable.

In [ ]:
async def _step(page):
    await page.goto(etherpad_url)
    await expect(page.locator(".hashview-search-box")).to_be_editable()

    global index_page
    index_page = page

await run_pw(_step)

## Create TestPad1 from the search box.

In [ ]:
import re

async def _step(page):
    await page.locator(".hashview-search-box").fill(pad1_title)
    await expect(page.locator('//button[text()="Create"]')).to_be_enabled()

    popup_future = page.wait_for_event('popup')
    await page.locator('//button[text()="Create"]').click()
    popup = await popup_future
    await expect(popup).to_have_title(re.compile(f"^{pad1_title}"))
    await expect(popup.locator('//iframe[@name="ace_outer"]')).to_be_visible(timeout=transition_timeout)

    global pad1_url
    pad1_url = popup.url
    return popup

await run_pw(_step)

## Append the hash tag to TestPad1.

In [ ]:
async def extract_ace_inner_docbody(page):
    iframe_locator = page.frame_locator('iframe[name="ace_outer"]').frame_locator('iframe[name="ace_inner"]')
    await expect(iframe_locator.locator('#innerdocbody')).to_be_visible(timeout=transition_timeout)
    return iframe_locator.locator('#innerdocbody')

async def _step(page):
    inner_docbody = await extract_ace_inner_docbody(page)
    await expect(inner_docbody).to_be_editable(timeout=transition_timeout)
    await inner_docbody.focus()
    await inner_docbody.press("End")
    for _ in range(2):
        await inner_docbody.press("Enter")
    await inner_docbody.type(hash_tag_original, delay=100)
    await expect(inner_docbody).to_contain_text(hash_tag_original, timeout=transition_timeout)

await run_pw(_step)

## Verify the rollup shows the create link for TestPad2.

In [ ]:
async def _step(page):
    hash_create_link = page.locator('.hash-link.hash-create .hash-title a')
    await expect(hash_create_link).to_have_text(pad2_title, timeout=transition_timeout)
    await hash_create_link.scroll_into_view_if_needed()

await run_pw(_step)

## Open TestPad2 from the rollup create link.

In [ ]:
async def _step(page):
    hash_create_link = page.locator('.hash-link.hash-create .hash-title a')
    await hash_create_link.click()
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{pad2_title}"))

    hash_link = page.locator('.hash-link:not(.hash-create) .hash-title a')
    await expect(hash_link).to_have_text(pad1_title, timeout=transition_timeout)
    await hash_link.scroll_into_view_if_needed()

    global pad2_url
    pad2_url = page.url

await run_pw(_step)

## Reload TestPad1 and ensure TestPad2 appears in the rollup.

In [ ]:
async def _step(page):
    await page.goto(pad1_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{pad1_title}"))

    hash_link = page.locator('.hash-link:not(.hash-create) .hash-title a')
    await expect(hash_link).to_have_text(pad2_title, timeout=transition_timeout)
    await hash_link.scroll_into_view_if_needed()

    await page.goto(pad2_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{pad2_title}"))

    hash_link = page.locator('.hash-link:not(.hash-create) .hash-title a')
    await expect(hash_link).to_have_text(pad1_title, timeout=transition_timeout)
    await hash_link.scroll_into_view_if_needed()

await run_pw(_step)

## Rename TestPad2 to TestPad3.

In [ ]:
import asyncio

async def _step(page):
    inner_docbody = await extract_ace_inner_docbody(page)
    await inner_docbody.focus()
    for _ in range(3):
        await inner_docbody.press("ArrowUp")
    await inner_docbody.press("End")
    for _ in range(len(pad2_title) + 1):
        await inner_docbody.press("Backspace")
    await inner_docbody.type(renamed_pad_title, delay=100)
    await inner_docbody.press("Enter")

    await page.wait_for_load_state('networkidle')
    await asyncio.sleep(1)  # Wait a moment to ensure the title change is processed
    await expect(page).to_have_title(re.compile(f"^{renamed_pad_title}"), timeout=transition_timeout)

    # Wait for the change title button to appear to ensure the title change is processed
    change_title_button = page.locator('.hashview-change-title')
    await expect(change_title_button).to_be_visible(timeout=transition_timeout)
    await expect(change_title_button).to_have_text(f'Title changed: {renamed_pad_title} from {pad2_title}', timeout=transition_timeout)

    # No links for renamed pad - ensure no old links are present
    old_hash_link = page.locator(f'.hash-link:not(.hash-create) .hash-title:has-text("{pad1_title}")')
    await expect(old_hash_link).to_have_count(0, timeout=transition_timeout)

    global pad2_url
    pad2_url = page.url

await run_pw(_step)

## Click the change title banner to update the hash link.

In [ ]:
import asyncio
import traceback

async def _step(page):
    change_title_button = page.locator('.hashview-change-title')
    await expect(change_title_button).to_be_visible(timeout=transition_timeout)
    await expect(change_title_button).to_contain_text(
        f'Title changed: {renamed_pad_title} from {pad2_title}', timeout=transition_timeout
    )
    await change_title_button.click()

    await expect(change_title_button).not_to_be_visible(timeout=transition_timeout)
    await page.wait_for_load_state('networkidle')

    # Retry until the link reappears
    for _ in range(5):
        try:
            await page.reload()
            hash_link = page.locator('.hash-link:not(.hash-create) .hash-title a')
            await expect(hash_link).to_have_text(pad1_title, timeout=transition_timeout)
            await hash_link.scroll_into_view_if_needed()
            break
        except:
            if _ == 4:
                raise
            print(f"Retrying after failure... ({_ + 1}/5)")
            traceback.print_exc()
            await asyncio.sleep(2)

await run_pw(_step)

## Confirm the hash in TestPad1 updates to #TestPad3.

In [ ]:
async def _step(page):
    await page.goto(pad1_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{pad1_title}"))

    inner_docbody = await extract_ace_inner_docbody(page)
    await expect(inner_docbody).to_contain_text(hash_tag_renamed, timeout=transition_timeout)
    await expect(inner_docbody).not_to_contain_text(hash_tag_original)

await run_pw(_step)

Clean up

In [ ]:
await finish_pw_context()

In [ ]:
!rm -fr {work_dir}